In [1]:
import re
import opencc
import pandas as pd

from tqdm import tqdm
from re import finditer

df = pd.read_excel("./corpus/_正體(繁體)/教育部重編國語辭典修訂本.xlsx")
df = df[["字詞名", "相似詞", "相反詞", "釋義"]]
df

,字詞名,相似詞,相反詞,釋義
0,八,NaN,NaN,[名]_x000D_\n1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」...
1,扒,NaN,NaN,[動]\n1.剝開。如：「把橘子扒開來吃。」\n2.扯掉。如：「扒衣裳」。\n3.挖開、刨開...
2,叭,NaN,NaN,[名]\n參見「喇叭」條。\n[擬]\n形容汽車、機車的喇叭聲。如：「忽聽叭的一聲，嚇了我一...
3,巴,NaN,NaN,[名]\n1.國名。春秋時所建的國家，故城約在今大陸地區四川省東部。\n2.量詞。計算大氣壓...
4,吧,NaN,NaN,[名]\n酒吧。為英文bar的音譯。如：「吧檯」、「吧女」。\n[動]\n吸（菸）。如：「他...
...,...,...,...,...
165538,瘜肉,NaN,NaN,即瘜。一種因黏膜異常而形成的突起物。多發生在鼻腔或腸道內。病因不明，有遺傳性和非遺傳性之分。...
165539,打扦,NaN,NaN,舊時男子向人請安時，左膝前屈，右腿後彎，上身稍向前俯，右手下垂，是一種介乎作揖、下跪之間的禮...
165540,八步床,NaN,NaN,一種有簷飾、帳幔和腳踏的舊式大床。參見「拔步床」條。《聊齋誌異》第一七回：「朝廷家有龍床，大...
165541,瀽,NaN,NaN,傾、倒。元．關漢卿〈溫太真玉鏡臺〉第一折：「我欲說話瀽無甚伎倆，把一瀽酒瀽一半在階基上。」元...


### 參見「xx」條

#### 只寫 `參見「xx」條。`

In [2]:
df[df['釋義'].str.contains(r"^參見「(.*)」條。$")]

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\2507547285.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['釋義'].str.contains(r"^參見「(.*)」條。$")]


,字詞名,相似詞,相反詞,釋義
42,鱍,NaN,NaN,參見「鱍鱍」條。
44,玻,NaN,NaN,參見「玻璃」條。
46,菠,NaN,NaN,參見「菠菜」條。
48,砵,NaN,NaN,參見「砵酒」條。
51,般,NaN,NaN,參見「般若」條。
...,...,...,...,...
157296,英尺,NaN,NaN,參見「呎」條。
163967,月蝕,NaN,NaN,參見「月食」條。
164912,雲南白藥,NaN,NaN,參見「白藥」條。
165492,屯,NaN,NaN,參見「屯留縣」條。


In [3]:
# 因為只寫 "參見「xx」條" 的字/詞語缺乏規律，故全部捨棄
df = df[~df['釋義'].str.contains(r"^參見「(.*)」條。$")]

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\3010003155.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['釋義'].str.contains(r"^參見「(.*)」條。$")]


#### 含有 `參見「xx」條。` 的釋義

In [4]:
df[df['釋義'].str.contains(r"參見「(.*)」條。")]

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\1190954975.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['釋義'].str.contains(r"參見「(.*)」條。")]


,字詞名,相似詞,相反詞,釋義
2,叭,NaN,NaN,[名]\n參見「喇叭」條。\n[擬]\n形容汽車、機車的喇叭聲。如：「忽聽叭的一聲，嚇了我一...
5,芭,NaN,NaN,[名]\n1.一種香草。《楚辭．屈原．九歌．禮魂》：「成禮兮會鼓，傳芭兮代舞。」宋．洪興祖．...
8,粑,NaN,NaN,[名]\n譯音用字。參見「糌粑」條。\n[綴]\n表乾扁的食物。如：「鍋粑」。
34,壩,NaN,NaN,[名]\n1.築在河上或湖口狹處，用以擋水並提高水位的建築物。《集韻．去聲．禡韻》：「壩，堰...
54,餑,NaN,NaN,[名]\n1.煮茶所得的精華。茶煮好後浮在表面上的泡，稱為「沫」。輕的沫稱為「花」，在湯中層...
...,...,...,...,...
165293,永嘉,NaN,NaN,1.縣名。參見「永嘉縣」條。\n2.漢朝沖帝的年號(西元145)。\n3.晉朝懷帝的年號（西...
165393,踊貴屨賤,NaN,NaN,比喻刑罰苛濫。參見「屨賤踊貴」條。
165470,用舍行藏,NaN,NaN,可以仕則仕，可以止則止，不強求富貴名利的處世態度。參見「用行舍藏」條。「舍」文獻異文作「捨」...
165499,北迴歸線,NaN,南回歸線,目前大多稱「北回歸線」。參見「北回歸線」條。


In [5]:
df[(df['字詞名'] == '屯') | (df['字詞名'] == '屯留縣')]

,字詞名,相似詞,相反詞,釋義
2875,屯,NaN,NaN,[動]\n1.聚集、儲存。如：「屯集」、「屯積」、「聚草屯糧」。《楚辭．屈原．離騷》：「屯車...
8124,屯,NaN,NaN,[名]\n1.困難、危難。《西遊記》第三七回：「你的災屯，想應天付，卻與我相類。」\n2.《...
55067,屯留縣,NaN,NaN,位於山西省東南的一縣。境內盛產米、麥、高粱、玉蜀黍、豆類，工業則以釀酒、製麵粉等業為主。


In [6]:
# 會有多個同字詞名是因為讀音不同，eg. https://dict.revised.moe.edu.tw/dictView.jsp?ID=8127&q=1&word=%E5%B1%AF#order2
# 這裡的處理方式是 groupby 並用 "\n" concat 釋義的 text
df['釋義_agg'] = df.groupby(['字詞名'])['釋義'].transform(lambda x: '\n'.join(x))
df = df[['字詞名', '相似詞', '相反詞', '釋義_agg']].drop_duplicates()
df = df.rename({'釋義_agg': '釋義'}, axis=1)

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\3381181161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['釋義_agg'] = df.groupby(['字詞名'])['釋義'].transform(lambda x: '\n'.join(x))


In [7]:
df[df['字詞名'] == '屯']

,字詞名,相似詞,相反詞,釋義
2875,屯,NaN,NaN,[動]\n1.聚集、儲存。如：「屯集」、「屯積」、「聚草屯糧」。《楚辭．屈原．離騷》：「屯車...


### 罕見字

In [8]:
# BTW, 全字庫 Unicode 範圍：20000~2FFFF, [\u20000-\U0002FFFF], ref: CNS11643 中文標準交換碼全字庫介紹, 下面被濾掉的也都在此範圍

# ref: https://blog.miniasp.com/post/2019/01/02/Common-Regex-patterns-for-Unicode-characters
# 中日韓統一表意文字列表 (CJK Unified Ideographs): 
# 「所有的漢字」都在這個範圍裡，包含正體中文、簡體中文與日文、韓文、越南文裡的漢字。
CJK_Unified_Ideographs = "\u4E00-\u9FFF"

# 中日韓統一表意文字擴展區 A (CJK Unified Ideographs Extension A): 
# 1997 年提出的擴充漢字，大多是罕見字，所有文字都不存在於 Big5 編碼中。有些俗稱的「難字」有可能會落在這區 Unicode 文字中。
CJK_Unified_Ideographs_Extension_A = "\u3400-\u4DBF"

unicode_pattern = rf'[{CJK_Unified_Ideographs} {CJK_Unified_Ideographs_Extension_A}]'
rarely_used_words_df = df[~df['字詞名'].str.contains(unicode_pattern)]

# 罕用字
rarely_used_words_df

,字詞名,相似詞,相反詞,釋義
85,_xDC5A__xD867_,NaN,NaN,(一)之讀音。\n[動]\n食物煮沸時，溢出湯汁。如：「米湯𩱚出來了，快熄火吧！」
190,_xDDBE__xD84D_,NaN,NaN,植物名。漆樹科漆樹屬，落葉喬木。樹披灰褐色而有赤褐色斑點。葉螺旋狀互生，奇數羽狀複葉。圓錐花...
640,_xDCA2__xD840_,NaN,NaN,[名]\n水的分流。《說文解字．𠂢部》：「𠂢，水之邪流別也。」
693,_xDD90__xD84F_,NaN,NaN,[動]\n用水浸物。通「泡」。如：「𣶐飯」。
1288,_xDE77__xD84D_,NaN,NaN,[名]\n𣙷果：植物名。漆樹科檬果屬，「芒果」之古稱。參見「芒果」條。
...,...,...,...,...
162990,_xDF46__xDBBF_,NaN,NaN,「羽」的異體字。
163460,_xDD82__xD87E_,NaN,NaN,「育」的異體字。
163775,&96f9._104_0.gif;,NaN,NaN,「鬱」的異體字。
164375,_xDEFE__xDB83_,NaN,NaN,「園」的異體字。


In [9]:
# 不採用含有罕用字的 row
df = df[df['字詞名'].str.contains(unicode_pattern)]

### 異體字 (VCC)

In [10]:
VCC_condition = (df['釋義'].str.contains("異體字")) & (df['字詞名'].str.len() == 1)

VCC_df = df[VCC_condition]
VCC_df

,字詞名,相似詞,相反詞,釋義
649,坏,NaN,NaN,[名]\n1.未經燒成的磚瓦陶器。《說文解字．土部》：「坏，一曰瓦未燒。」如：「土坏」、「陶...
758,逄,NaN,NaN,[名]\n姓。如周代有逄同，漢代有逄安。\n「逢」的異體字。
1070,袜,NaN,NaN,[名]\n婦女所使用的抹胸、肚兜。隋．煬帝〈喜春遊歌〉：「錦袖淮南舞，寶袜楚宮腰。」也稱為「...
1307,懞,NaN,NaN,[形]\n樸實忠厚。《管子．五輔》：「敦懞純固，以備禍亂。」《西遊記》第二三回：「那獃子雖是...
1379,糸,NaN,NaN,[名]\n1.細絲。《說文解字．糸部》：「糸，細絲也。」\n2.二一四部首之一。\n「絲」的...
...,...,...,...,...
165067,韵,NaN,NaN,「韻」的異體字。
165173,愠,NaN,NaN,「慍」的異體字。
165188,藴,NaN,NaN,「蘊」的異體字。
165340,咏,NaN,NaN,「詠」的異體字。


In [11]:
# only_VCC: 只含有異體字的說明
only_VCC_condition = (VCC_df['釋義'].str.contains(r'(^「(.*)」的異體字。)|(^同「(.*)」。$)'))
only_VCC_df = VCC_df[only_VCC_condition]

only_VCC_df

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\1775041063.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  only_VCC_condition = (VCC_df['釋義'].str.contains(r'(^「(.*)」的異體字。)|(^同「(.*)」。$)'))


,字詞名,相似詞,相反詞,釋義
12622,罢,NaN,NaN,「罷」的異體字。
12647,覇,NaN,NaN,「霸」的異體字。
12687,垻,NaN,NaN,「壩」的異體字。
12929,盋,NaN,NaN,「缽」的異體字。
12930,鉢,NaN,NaN,「缽」的異體字。
...,...,...,...,...
165067,韵,NaN,NaN,「韻」的異體字。
165173,愠,NaN,NaN,「慍」的異體字。
165188,藴,NaN,NaN,「蘊」的異體字。
165340,咏,NaN,NaN,「詠」的異體字。


In [12]:
# not_only_VCC: 怎麼處理異體字中有多重詞性的情況 ? => 先挑出來，等到 "拆分有多詞性的字" 區段時再做處理
not_only_VCC_df = VCC_df[~only_VCC_condition]

not_only_VCC_df

,字詞名,相似詞,相反詞,釋義
649,坏,NaN,NaN,[名]\n1.未經燒成的磚瓦陶器。《說文解字．土部》：「坏，一曰瓦未燒。」如：「土坏」、「陶...
758,逄,NaN,NaN,[名]\n姓。如周代有逄同，漢代有逄安。\n「逢」的異體字。
1070,袜,NaN,NaN,[名]\n婦女所使用的抹胸、肚兜。隋．煬帝〈喜春遊歌〉：「錦袖淮南舞，寶袜楚宮腰。」也稱為「...
1307,懞,NaN,NaN,[形]\n樸實忠厚。《管子．五輔》：「敦懞純固，以備禍亂。」《西遊記》第二三回：「那獃子雖是...
1379,糸,NaN,NaN,[名]\n1.細絲。《說文解字．糸部》：「糸，細絲也。」\n2.二一四部首之一。\n「絲」的...
1727,丰,NaN,NaN,[形]\n1.草木茂盛的樣子。《說文解字．生部》：「丰，艸盛丰丰也。」\n2.容貌美好或豐滿...
1909,荅,NaN,NaN,[名]\n小豆。《說文解字．艸》：「荅，小尗也。」清．王念孫《廣雅疏證．卷十上．釋草》：「小...
2059,党,NaN,NaN,[名]\n姓。如宋代有党進。\n「黨」的異體字。
2467,台,NaN,NaN,大陸地區浙江省台州、天台山等地之台的舊讀。\n[動]\n喜悅。通「怡」。《史記．卷一三○．太...
2834,媠,NaN,NaN,[形]\n美好。如：「婑媠」。《文選．曹植．七啟》：「收亂髮兮拂蘭澤，行媠服兮揚幽若。」李善...


In [13]:
# extract VCCs
only_VCC_pattern1_df = only_VCC_df['釋義'].str.extract(r'「(.*)」的異體字。').rename({0: 'value'}, axis='columns').dropna()
only_VCC_pattern2_df = only_VCC_df['釋義'].str.extract(r'同「(.*)」。').rename({0: 'value'}, axis='columns').dropna()
only_VCC_value_df = pd.concat([only_VCC_pattern1_df, only_VCC_pattern2_df])

only_VCC_and_value_df = pd.merge(only_VCC_df, only_VCC_value_df, left_index=True, right_index=True)
only_VCC_and_value_df

,字詞名,相似詞,相反詞,釋義,value
12622,罢,NaN,NaN,「罷」的異體字。,罷
12647,覇,NaN,NaN,「霸」的異體字。,霸
12687,垻,NaN,NaN,「壩」的異體字。,壩
12929,盋,NaN,NaN,「缽」的異體字。,缽
12930,鉢,NaN,NaN,「缽」的異體字。,缽
...,...,...,...,...,...
165067,韵,NaN,NaN,「韻」的異體字。,韻
165173,愠,NaN,NaN,「慍」的異體字。,慍
165188,藴,NaN,NaN,「蘊」的異體字。,蘊
165340,咏,NaN,NaN,「詠」的異體字。,詠


In [14]:
Variant_Chinese_characters_instruction_dataset = pd.DataFrame()

Variant_Chinese_characters_instruction_dataset["input"] = only_VCC_and_value_df['value'].tolist()
Variant_Chinese_characters_instruction_dataset["output"] = only_VCC_and_value_df['字詞名'].tolist()
Variant_Chinese_characters_instruction_dataset["category"] = "translate_from_VCC" # Variant Chinese Characters (VCC) = 異體字
Variant_Chinese_characters_instruction_dataset["instruction"] = "這個字的異體字是什麼"

Variant_Chinese_characters_instruction_dataset = \
    Variant_Chinese_characters_instruction_dataset[["category", "instruction", "input", "output"]]

Variant_Chinese_characters_instruction_dataset

,category,instruction,input,output
0,translate_from_VCC,這個字的異體字是什麼,罷,罢
1,translate_from_VCC,這個字的異體字是什麼,霸,覇
2,translate_from_VCC,這個字的異體字是什麼,壩,垻
3,translate_from_VCC,這個字的異體字是什麼,缽,盋
4,translate_from_VCC,這個字的異體字是什麼,缽,鉢
...,...,...,...,...
1176,translate_from_VCC,這個字的異體字是什麼,韻,韵
1177,translate_from_VCC,這個字的異體字是什麼,慍,愠
1178,translate_from_VCC,這個字的異體字是什麼,蘊,藴
1179,translate_from_VCC,這個字的異體字是什麼,詠,咏


In [15]:
# 去除釋義中只含有異體字的 row, not_only_VCC_df 繼續留在 df 內等下個 part 再處理
df = df.loc[~df.index.isin(VCC_df[only_VCC_condition].index)]
df

,字詞名,相似詞,相反詞,釋義
0,八,NaN,NaN,[名]_x000D_\n1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」...
1,扒,NaN,NaN,[動]\n1.剝開。如：「把橘子扒開來吃。」\n2.扯掉。如：「扒衣裳」。\n3.挖開、刨開...
2,叭,NaN,NaN,[名]\n參見「喇叭」條。\n[擬]\n形容汽車、機車的喇叭聲。如：「忽聽叭的一聲，嚇了我一...
3,巴,NaN,NaN,[名]\n1.國名。春秋時所建的國家，故城約在今大陸地區四川省東部。\n2.量詞。計算大氣壓...
4,吧,NaN,NaN,[名]\n酒吧。為英文bar的音譯。如：「吧檯」、「吧女」。\n[動]\n吸（菸）。如：「他...
...,...,...,...,...
165538,瘜肉,NaN,NaN,即瘜。一種因黏膜異常而形成的突起物。多發生在鼻腔或腸道內。病因不明，有遺傳性和非遺傳性之分。...
165539,打扦,NaN,NaN,舊時男子向人請安時，左膝前屈，右腿後彎，上身稍向前俯，右手下垂，是一種介乎作揖、下跪之間的禮...
165540,八步床,NaN,NaN,一種有簷飾、帳幔和腳踏的舊式大床。參見「拔步床」條。《聊齋誌異》第一七回：「朝廷家有龍床，大...
165541,瀽,NaN,NaN,傾、倒。元．關漢卿〈溫太真玉鏡臺〉第一折：「我欲說話瀽無甚伎倆，把一瀽酒瀽一半在階基上。」元...


In [16]:
df.loc[351]

字詞名                 秘
相似詞               NaN
相反詞               NaN
釋義     (一)之又音。\n同「祕」。
Name: 351, dtype: object

### 拆分有多詞性的字

In [17]:
# 有多個詞性解釋的
POS_pattern = r"\[(名|動|形|副|擬|代|助|連|歎)\]"
multi_POS_df = df[df['釋義'].str.contains(POS_pattern)]
multi_POS_df

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\1128347670.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  multi_POS_df = df[df['釋義'].str.contains(POS_pattern)]


,字詞名,相似詞,相反詞,釋義
0,八,NaN,NaN,[名]_x000D_\n1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」...
1,扒,NaN,NaN,[動]\n1.剝開。如：「把橘子扒開來吃。」\n2.扯掉。如：「扒衣裳」。\n3.挖開、刨開...
2,叭,NaN,NaN,[名]\n參見「喇叭」條。\n[擬]\n形容汽車、機車的喇叭聲。如：「忽聽叭的一聲，嚇了我一...
3,巴,NaN,NaN,[名]\n1.國名。春秋時所建的國家，故城約在今大陸地區四川省東部。\n2.量詞。計算大氣壓...
4,吧,NaN,NaN,[名]\n酒吧。為英文bar的音譯。如：「吧檯」、「吧女」。\n[動]\n吸（菸）。如：「他...
...,...,...,...,...
11927,禜,NaN,NaN,[名]\n古代一種祭拜日月、星辰、山川、風雨等的祭祀，以祈求消災除禍。《說文解字．示部》：「...
11928,醟,NaN,NaN,[副]\n酒後亂性的樣子。《晉書．卷五○．曹志等傳．贊曰》：「謀甫三爵，酗醟斯作。」
79927,海里,NaN,NaN,[名]\n量詞。計算海面距離的單位。一海里約為一千八百五十二公尺。也稱為「浬」。
111398,褶子,NaN,NaN,[名]\n動物名： (1)雀形目。似鵲，毛色黑而有文采，嘴、腳為紅色，尾巴長，性凶悍。也稱為...


In [18]:
df[df['釋義'].str.contains(r"\[助\]")]

,字詞名,相似詞,相反詞,釋義
4,吧,NaN,NaN,[名]\n酒吧。為英文bar的音譯。如：「吧檯」、「吧女」。\n[動]\n吸（菸）。如：「他...
43,波,NaN,NaN,[名]\n1.水因湧流或風力振盪所產生的起伏現象。如：「水波」、「海波」。\n2.比喻目光。...
45,啵,NaN,NaN,[動]\n親吻。如：「啵一下臉頰」。\n[助]\n表示商榷或祈使的語尾助詞。如：「弟兄們，努...
141,唄,NaN,NaN,[名]\n印度詠經或唱讚都稱為「唄」，在中國只有唱讚稱為「唄」。南朝梁．簡文帝〈吳郡石像碑〉...
184,被,NaN,NaN,[名]_x000D_\n1.被子。睡眠時蓋在身上的東西。《楚辭．宋玉．招魂》：「翡翠珠被，爛...
...,...,...,...,...
11745,曰,NaN,NaN,[動]\n1.說。用於文言文。《論語．學而》：「子曰：『學而時習之，不亦說乎？』」《孟子．梁...
11752,越,NaN,NaN,[動]\n1.度過、跨過。如：「翻山越嶺」、「越牆而走」。《楚辭．屈原．天問》：「阻窮西征，...
11762,粵,NaN,NaN,[助]\n1.句首發語詞，無義。《漢書．卷八四．翟方進傳》：「粵其聞日，宗室之儁有四百人，民...
11798,爰,NaN,NaN,[連]\n於是。《書經．無逸》：「作其即位，爰知小人之依，能保惠于庶民。」《文選．張衡．思玄...


In [19]:
multi_POS_df[multi_POS_df['字詞名'].str.len() != 1]

,字詞名,相似詞,相反詞,釋義
79927,海里,NaN,NaN,[名]\n量詞。計算海面距離的單位。一海里約為一千八百五十二公尺。也稱為「浬」。
111398,褶子,NaN,NaN,[名]\n動物名： (1)雀形目。似鵲，毛色黑而有文采，嘴、腳為紅色，尾巴長，性凶悍。也稱為...


In [20]:
def split_to_lst(string: str):
    """Split string by seperator.
    Args:
        string: str, string to be split,
            eg1. "[名]\n量詞。計算海面距離的單位。"
            eg2. "\n[動]喘氣。同「喘」。"
    Returns:
        eg1. ['[名]', '量詞。計算海面距離的單位。']
        eg2. ['[動]', '喘氣。同「喘」。']
    """
    no_newline_POS_regex = POS_pattern + "[^\n]" # expected [pos]\nxx but some [pos]xx are exist.

    if re.findall(no_newline_POS_regex, string): # [pos] 後沒有 \n
        for match in finditer(no_newline_POS_regex, string):
            # print(match.span(), match.group())
            start = match.span()[0]
            string = string[:start+3] + '\n' + string[start+3:] # Cannot replace with dict easily, let's do it dirty...

    lst = re.split('[_x000D_\n]', string)
    return [i for i in lst if i] # remove empty elements

In [21]:
def convert_interpretion_lst_to_dict(interpretion_lst):
    """convert interpretion list to dict.
    Args:
        interpretion_lst : list
        eg. 
            [
                '[名]',
                '1.水因湧流或風力振盪所產生的起伏現象。如：「水波」、「海波」。',
                '2.比喻目光。如：「眼波」、「秋波」。...
                '[動]',
                '1.跑。如：「奔波」。明．李翊《俗呼小錄》卷一：「跑謂之波。」', ...
            ]
    Returns:
        eg.
            {
                '[名]': '1.水因湧流或風力振盪所產生的起伏現象。如：「水波」、「海波」。2.比喻目光。如：「眼波」、「秋波」...。
                '[動]': '1.跑。如：「奔波」。明．李翊《俗呼小錄》卷一：「跑謂之波。」,...
            }
    """
    d = dict()
    pos = None # 詞性

    for i in interpretion_lst:
        # print(i)
        if re.match(POS_pattern, i):
            d[i] = ""
            pos = i
        elif not re.match(POS_pattern, i) and not d: # 防止 ['xxx', '[名]', '1. ...'] 這種字串在 pos type 前的 case
            pass
        elif re.match('[\( | （]',  i): # 不採用含有括號的 element，因為通常是標註讀音
            pass
        elif '(一)之又音' in i: # 不會使用到讀音，故跳過
            pass
        else:
            d[pos] += i

    return d


In [22]:
split_to_lst(df.loc[4, "釋義"])

['[名]',
 '酒吧。為英文bar的音譯。如：「吧檯」、「吧女」。',
 '[動]',
 '吸（菸）。如：「他吧了一口菸，緩和一下情緒。」',
 '[擬]',
 '形容短暫的爆裂聲。如：「吧！吧！吧！打了三槍。」',
 '[助]',
 '1.用於句末：(1)表示商量、請求等語氣。如：「給我吧！」(2)表示指使。如：「快走吧！」(3)表示同意。如：「好吧！」(4)表示推測。如：「明天該不會下雨吧？」(5)表示感嘆。如：「算了吧！」',
 '2.用於句中，表示停頓。只用於對舉的句型中，表示猶疑、兩難的語氣。如：「做吧，沒有工夫；不做吧，又急著要用。」']

In [23]:
def extract_classical_chinese_example(string: str):
    """Extract allusion from string.
    Args:
        string: str, string to extract,
            eg1. 奔波」。明．李翊《俗呼小錄》卷一：「跑謂之波。」
            eg2. 秋波」。《文選．宋玉．神女賦》：「望余帷而延視兮，若流波之將瀾。」
    Returns:
        eg1. ['奔波', '明．李翊', '俗呼小錄》卷一：跑謂之波。']
        eg2. ['眼波', '秋波', '', '文選．宋玉．神女賦》：望余帷而延視兮，若流波之將瀾。」']
    """
    sp = re.split(r'」。(.*)《', string)
    lst = []
    for s in sp:
        if "》" in s:
            s1 = '《' + s
        else:
            s1 = re.sub(r'[「」]', '', s)
        lst.append(s1)

    return lst

In [24]:
def extract_usage_example(sentence: str):
    """Extract allusion from string.
    Args:
        string: str, string to extract,
            eg1. 奔波」。明．李翊《俗呼小錄》卷一：「跑謂之波。」
            eg2. 秋波」。《文選．宋玉．神女賦》：「望余帷而延視兮，若流波之將瀾。」
    Returns:
        eg1. ['奔波', '明．李翊', '俗呼小錄》卷一：跑謂之波。']
        eg2. ['眼波', '秋波', '', '文選．宋玉．神女賦》：望余帷而延視兮，若流波之將瀾。」']
    """
    return re.findall('(.*)。如：「(.*)。?」。?', sentence)

In [25]:
t2s_converter = opencc.OpenCC('t2s.json')

"""traditional chinese to simplified chinese"""
def convert_TC_to_SC(zhtw_text: str, converter=None):
    if not converter:
        converter = opencc.OpenCC('t2s.json')
    return converter.convert(zhtw_text)

In [26]:
def convert_pos_key_to_pos_type(pos_key: str):
    """Convert pos key to pos type, support pos key: \[(名|動|形|副|擬|代|助|連|歎)\]
    Args:
        pos_key: str,
            eg1. "[名]"
            eg2. "[形]"
    Returns:
        eg1. "名詞"
        eg2. "形容詞"
    """
    POS_mapping = {
        "[名]": "名詞",
        "[動]": "動詞",
        "[形]": "形容詞",
        "[副]": "副詞",
        "[擬]": "擬聲詞",
        "[代]": "代名詞",
        "[助]": "助詞",
        "[連]": "連接詞",
        "[歎]": "感嘆詞"
    }

    return POS_mapping[pos_key.strip()]

In [27]:
WORD_COL = "字詞名"
INTERPRETION_COL = "釋義"

# row = df.loc[43]
row = df.loc[0]
# df.loc[0]['釋義'] = df.loc[0]['釋義'].replace('_x000D_\n（「八」字口語連用在去聲字前讀成陽平，如：「八號」、「八拜」。）', '')

word_name = row["字詞名"]
interpretion_lst = split_to_lst(row["釋義"])

interpretion_dict = convert_interpretion_lst_to_dict(interpretion_lst)
interpretion_dict

{'[名]': '1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」，阿拉伯數字作「8」。2.姓。如漢代有西域人八滑。3.二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面。如：「四通八達」。'}

In [28]:
"""Instructions"""
interpret_instruction_with_example = "「{}」裡面的「{}」是什麼意思" # eg. 「吧！吧！吧！打了三槍。」裡面的「吧」是什麼意思

pos_instruction_informal = "「{}」什麼時候可以做為這個詞性使用" # 詞性任務

translate_from_SC_instruction = "翻譯成中文"

In [29]:
INSTRUCTION_DATASET_COLS = ['category', 'instruction', 'input', 'output']

In [30]:
WORD_COL = "字詞名"
INTERPRETION_COL = "釋義"

interpret_instructions = []
interpret_instruction_dataset = pd.DataFrame(columns=INSTRUCTION_DATASET_COLS)

for index, row in tqdm(multi_POS_df.iterrows(), total=multi_POS_df.shape[0]):
    word_name = row[WORD_COL]

    interpretion_lst = split_to_lst(row[INTERPRETION_COL])
    interpretion_dict = convert_interpretion_lst_to_dict(interpretion_lst)

    # 分割例句以便做為 interpret 的 instruction
    for pos_key, value in interpretion_dict.items():
        pos_type = convert_pos_key_to_pos_type(pos_key) # "[名]" -> "名詞"

        if re.match('\d+\.', value): # 有列點的釋義，eg. 1.跑。如：「奔波」。
            sentence_lst = re.split(r'\d+\.', value)

            for sentence in sentence_lst:
                temp_lst = []

                examples_lst = []
                classical_chinese_example_author = ""
                classical_chinese_example_sentence = ""

                matched = extract_usage_example(sentence) # eg. [('水因湧流或風力振盪所產生的起伏現象', '水波」、「海波')]

                if matched:
                    interpretion = matched[0][0]
                    ex = matched[0][1]

                    # 通常都是用 "、" 分隔多種例子，且要避免拆分「六、七、八、九……」這種例子
                    if ('、' in ex) and (('」、' in ex) or ('、「' in ex)): 
                        l = ex.split('、')
                    else:
                        l = [ex]

                    for i in l :
                        temp_lst.extend(extract_classical_chinese_example(i))

                    # print(temp_lst)

                    have_classical_chinese_example = False

                    for i in range(1, len(temp_lst)):
                        if '《' in temp_lst[i]:
                            have_classical_chinese_example = True
                            classical_chinese_example_author = temp_lst[i-1]
                            ex_lst = temp_lst[:i-1]
                            classical_chinese_example_sentence = temp_lst[i]

                    if have_classical_chinese_example: # 有古典文獻例證
                        # NOTE: 用不到作者故合併
                        classical_chinese_example_sentence = classical_chinese_example_author + classical_chinese_example_sentence

                        # print(classical_chinese_example_sentence)
                        # print(ex_lst)

                        # Modern Chinese 現代中文例句
                        for ex in ex_lst:
                            MC_instruction = interpret_instruction_with_example.format(ex, word_name)
                            _output = f"{pos_type}，" + interpretion

                            interpret_instructions.append({
                                "category": "interpret",
                                "instruction": MC_instruction, 
                                "input": "",
                                "output": _output
                            })

                        # Classical Chinese 文言文例句
                        CC_instruction = interpret_instruction_with_example.format(classical_chinese_example_sentence, word_name)
                        _output = f"{pos_type}，" + interpretion

                        interpret_instructions.append({
                            "category": "interpret",
                            "instruction": CC_instruction, 
                            "input": "",
                            "output": _output
                        })

                    else:
                        # 現代中文例句
                        for ex in temp_lst:
                            MC_instruction = interpret_instruction_with_example.format(ex, word_name)
                            _output = f"{pos_type}，" + interpretion

                            interpret_instructions.append({
                                "category": "interpret",
                                "instruction": MC_instruction, 
                                "input": "",
                                "output": _output
                            })

                    # print(MC_instruction, _output)

                elif not matched: # 有列點的釋例但沒有抓到 pattern 以及整句的列點釋例 -> 直接當成翻譯素材
                    interpret_instructions.append({
                        "category": "translate_from_SC",
                        "instruction": translate_from_SC_instruction, 
                        "input": convert_TC_to_SC(value),
                        "output": value
                    })

            else: # 沒有列點的釋例 -> 直接當成翻譯素材
                interpret_instructions.append({
                    "category": "translate_from_SC",
                    "instruction": translate_from_SC_instruction, 
                    "input": convert_TC_to_SC(sentence),
                    "output": sentence
                })

        else: # 單句釋義，eg. 元曲科白中常用作語助詞
            interpret_instructions.append({
                "category": "POS",
                "instruction": pos_instruction_informal.format(word_name), 
                "input": pos_type,
                "output": value
            })

    interpret_instruction_dataset = pd.concat([interpret_instruction_dataset, pd.DataFrame.from_records(interpret_instructions)])
    interpret_instruction_dataset.drop_duplicates(inplace=True)


  0%|          | 16/8780 [00:00<03:48, 38.37it/s]

100%|██████████| 8780/8780 [14:04<00:00, 10.39it/s]


In [31]:
interpret_instruction_dataset

,category,instruction,input,output
0,translate_from_SC,翻譯成中文,1.介于七与九之间的自然数。如：「六、七、八、九……」。大写作「捌」，阿拉伯数字作「8」。2...,1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」，阿拉伯數字作「8」。2...
1,interpret,「六、七、八、九……。大寫作捌，阿拉伯數字作8」裡面的「八」是什麼意思,,名詞，介於七與九之間的自然數
3,interpret,「四通八達」裡面的「八」是什麼意思,,名詞，二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面
4,translate_from_SC,翻譯成中文,二一四部首之一。[形]表示数量是八的。如：「八字」、「八方」。[副]形容多数或多方面。如：「...,二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面。如：「...
5,translate_from_SC,翻譯成中文,1.掘、挖。如：「扒土」、「扒洞」、「他在地里扒出土豆儿来。」2.拾。《西游记》第一四回：「...,1.掘、挖。如：「扒土」、「扒洞」、「他在地裡扒出土豆兒來。」2.拾。《西遊記》第一四回：「...
...,...,...,...,...
49784,POS,「醟」什麼時候可以做為這個詞性使用,副詞,酒後亂性的樣子。《晉書．卷五○．曹志等傳．贊曰》：「謀甫三爵，酗醟斯作。」
49785,POS,「海里」什麼時候可以做為這個詞性使用,名詞,量詞。計算海面距離的單位。一海里約為一千八百五十二公尺。也稱為「浬」。
49786,POS,「褶子」什麼時候可以做為這個詞性使用,名詞,動物名： (1)雀形目。似鵲，毛色黑而有文采，嘴、腳為紅色，尾巴長，性凶悍。也稱為「山鵲」。...
49787,translate_from_SC,翻譯成中文,1.竹皮。「篛」的本字。《说文解字．竹部》：「箬，楚谓竹皮曰箬。」2.竹名。秆细柱形，高约一...,1.竹皮。「篛」的本字。《說文解字．竹部》：「箬，楚謂竹皮曰箬。」2.竹名。稈細柱形，高約一...


In [32]:
# 只有一種詞性或單句解釋的
single_POS_df = df[~df[INTERPRETION_COL].str.contains(POS_pattern)]
single_POS_df

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\3937920975.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  single_POS_df = df[~df[INTERPRETION_COL].str.contains(POS_pattern)]


,字詞名,相似詞,相反詞,釋義
37,琶,NaN,NaN,(一)之又音。\n(一)之語音。
79,荸,NaN,NaN,(一)之又音。
84,鵓,NaN,NaN,參見「鵓鴿」、「鵓鴣」等條。
100,礴,NaN,NaN,參見「槃礴」、「磅礴」等條。
105,蔔,NaN,NaN,(二)之讀音。
...,...,...,...,...
165538,瘜肉,NaN,NaN,即瘜。一種因黏膜異常而形成的突起物。多發生在鼻腔或腸道內。病因不明，有遺傳性和非遺傳性之分。...
165539,打扦,NaN,NaN,舊時男子向人請安時，左膝前屈，右腿後彎，上身稍向前俯，右手下垂，是一種介乎作揖、下跪之間的禮...
165540,八步床,NaN,NaN,一種有簷飾、帳幔和腳踏的舊式大床。參見「拔步床」條。《聊齋誌異》第一七回：「朝廷家有龍床，大...
165541,瀽,NaN,NaN,傾、倒。元．關漢卿〈溫太真玉鏡臺〉第一折：「我欲說話瀽無甚伎倆，把一瀽酒瀽一半在階基上。」元...


In [33]:
meaningless_sentence_regex = r'(參見「\w+」(條|.*)?。)|(\(\w+\)之\w+音。)|(同「(.*)」。)'

single_POS_df[INTERPRETION_COL] = single_POS_df[INTERPRETION_COL] \
    .replace(to_replace=meaningless_sentence_regex, value='', regex=True) \
    .replace(to_replace='\n', value='', regex=True)
single_POS_df = single_POS_df[(single_POS_df[INTERPRETION_COL] != "")]

single_POS_df

C:\Users\T160\AppData\Local\Temp\ipykernel_7244\995438967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_POS_df[INTERPRETION_COL] = single_POS_df[INTERPRETION_COL] \


,字詞名,相似詞,相反詞,釋義
305,泵,NaN,NaN,一種用以增加液體的壓力，用來移動液體的裝置，即是對流體作功的機械。也稱為「幫浦」。
874,氕,NaN,NaN,氫的最輕同位素。質量數1，原子由一個質子和一個電子組成。普通的氫中含有百分之九十九點九八的氕...
1092,貘,NaN,NaN,1.動物名。古籍中的食鐵獸。一說為白豹。《說文解字．豸部》：「貘，似熊而黃黑色，出蜀中。」《...
1164,茆,NaN,NaN,通「茅」。禾本科白茅屬，「茅」、「白茅」之古稱，
1390,峚,NaN,NaN,山名。即峚山。約位在陝西省商洛市商南縣境內。
...,...,...,...,...
165538,瘜肉,NaN,NaN,即瘜。一種因黏膜異常而形成的突起物。多發生在鼻腔或腸道內。病因不明，有遺傳性和非遺傳性之分。...
165539,打扦,NaN,NaN,舊時男子向人請安時，左膝前屈，右腿後彎，上身稍向前俯，右手下垂，是一種介乎作揖、下跪之間的禮...
165540,八步床,NaN,NaN,一種有簷飾、帳幔和腳踏的舊式大床。《聊齋誌異》第一七回：「朝廷家有龍床，大人家有八步床、頂子...
165541,瀽,NaN,NaN,傾、倒。元．關漢卿〈溫太真玉鏡臺〉第一折：「我欲說話瀽無甚伎倆，把一瀽酒瀽一半在階基上。」元...


In [34]:
interpret_instruction_word = "解釋這個詞"

In [35]:
# 處理單句釋義，interpret + translate_from_SC
instruction_dataset1 = pd.DataFrame(columns=INSTRUCTION_DATASET_COLS)
instruction_dataset2 = pd.DataFrame(columns=INSTRUCTION_DATASET_COLS)

instruction_dataset1['input'] = single_POS_df[WORD_COL].values
instruction_dataset1["output"] = single_POS_df[INTERPRETION_COL].values
instruction_dataset1.loc[:, 'category'] = 'interpret'
instruction_dataset1.loc[:, "instruction"] = interpret_instruction_word

instruction_dataset2['input'] = single_POS_df[INTERPRETION_COL].apply(lambda x: convert_TC_to_SC(x))
instruction_dataset2["output"] = single_POS_df[INTERPRETION_COL].values
instruction_dataset2.loc[:, 'category'] = 'translate_from_SC'
instruction_dataset2.loc[:, "instruction"] = translate_from_SC_instruction

instruction_dataset = pd.concat([instruction_dataset1, instruction_dataset2])

MOE_instruction_dataset = pd.concat([interpret_instruction_dataset, instruction_dataset])

In [36]:
MOE_instruction_dataset

,category,instruction,input,output
0,translate_from_SC,翻譯成中文,1.介于七与九之间的自然数。如：「六、七、八、九……」。大写作「捌」，阿拉伯数字作「8」。2...,1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」，阿拉伯數字作「8」。2...
1,interpret,「六、七、八、九……。大寫作捌，阿拉伯數字作8」裡面的「八」是什麼意思,,名詞，介於七與九之間的自然數
3,interpret,「四通八達」裡面的「八」是什麼意思,,名詞，二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面
4,translate_from_SC,翻譯成中文,二一四部首之一。[形]表示数量是八的。如：「八字」、「八方」。[副]形容多数或多方面。如：「...,二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面。如：「...
5,translate_from_SC,翻譯成中文,1.掘、挖。如：「扒土」、「扒洞」、「他在地里扒出土豆儿来。」2.拾。《西游记》第一四回：「...,1.掘、挖。如：「扒土」、「扒洞」、「他在地裡扒出土豆兒來。」2.拾。《西遊記》第一四回：「...
...,...,...,...,...
165538,translate_from_SC,翻譯成中文,即瘜。一种因黏膜异常而形成的突起物。多发生在鼻腔或肠道内。病因不明，有遗传性和非遗传性之分。...,即瘜。一種因黏膜異常而形成的突起物。多發生在鼻腔或腸道內。病因不明，有遺傳性和非遺傳性之分。...
165539,translate_from_SC,翻譯成中文,旧时男子向人请安时，左膝前屈，右腿后弯，上身稍向前俯，右手下垂，是一种介乎作揖、下跪之间的礼...,舊時男子向人請安時，左膝前屈，右腿後彎，上身稍向前俯，右手下垂，是一種介乎作揖、下跪之間的禮...
165540,translate_from_SC,翻譯成中文,一种有簷饰、帐幔和脚踏的旧式大床。《聊斋志异》第一七回：「朝廷家有龙床，大人家有八步床、顶子...,一種有簷飾、帳幔和腳踏的舊式大床。《聊齋誌異》第一七回：「朝廷家有龍床，大人家有八步床、頂子...
165541,translate_from_SC,翻譯成中文,倾、倒。元．关汉卿〈温太真玉镜台〉第一折：「我欲说话瀽无甚伎俩，把一瀽酒瀽一半在阶基上。」元...,傾、倒。元．關漢卿〈溫太真玉鏡臺〉第一折：「我欲說話瀽無甚伎倆，把一瀽酒瀽一半在階基上。」元...


### 相似詞 & 相反詞

In [37]:
SYNONYM_COL = "近義詞"
ANTONYM_COL = "反義詞"

have_synonym_or_antonym_df = df[(~df['相似詞'].isna()) | (~df['相反詞'].isna())] # 有相似詞 or 有相反詞
have_synonym_or_antonym_df = have_synonym_or_antonym_df.rename({'相似詞': SYNONYM_COL, '相反詞': ANTONYM_COL}, axis='columns')

have_synonym_or_antonym_df

,字詞名,近義詞,反義詞,釋義
11931,八百壯士,四行孤軍,NaN,民國二十六年八月十三日，日軍大舉入侵淞、滬。我軍第八十八師團長謝晉元率領八百名戰士，孤軍苦守...
11955,八面玲瓏,八面見光、面面俱到、面面俱圓、左右逢源,四處碰壁、處處碰壁,1.屋子四面八方敞亮通明。宋．葛長庚〈滿江紅．鈞天高處〉詞：「八面玲瓏光不夜，四圍晃耀寒如月...
11957,八面見光,八面玲瓏、左右逢源,處處碰壁,形容非常圓滑，善於應付。《兒女英雄傳》第一○回：「張姑娘這幾句話，說得軟中帶硬，八面兒見光，...
11958,八面受敵,四面楚歌,NaN,比喻處境十分困難、危險。如：「你平常處處得罪人，如今八面受敵，看你如何自處？」也作「八方受敵」。
11959,八面威風,NaN,垂頭喪氣,1.比喻聲勢浩大、神氣十足的樣子。元．鄭光祖《三戰呂布》第三折：「托賴著真天子百靈咸助，大將...
...,...,...,...,...
165482,用武,NaN,媾和,1.用兵，使用武力。《老殘遊記》第一九回：「許亮說：『難道吳二爺騙你，我許大爺也會騙你嗎？』...
165487,佣鈿,回佣,NaN,買賣貨物時，中間人或仲介人所得的金錢。也稱為「用錢」。
165489,佣錢,回佣,NaN,買賣貨物時，中間人或仲介人所得的金錢。也稱為「用錢」。
165499,北迴歸線,NaN,南回歸線,目前大多稱「北回歸線」。參見「北回歸線」條。


In [38]:
# 如果之後有時間改成 python script 的話可以順便 refactor, 參考 1.2 的 tasks_instructions & task_category_mappings
tasks_instructions = {
    '近義詞': "舉出{}個近似詞",
    '反義詞': "舉出{}個反義詞"
}

task_category_mappings = {
    '近義詞': 'synonym',
    '反義詞': 'antonym'
}

In [39]:
def split_and_get_len(string, sep):
    return str(int(len(string.split(sep))))

In [40]:
SEP = '、'

for index, row in tqdm(have_synonym_or_antonym_df.iterrows(), total=have_synonym_or_antonym_df.shape[0]):
    synonym_and_antonym_instructions = []

    word_name = row[WORD_COL]
    synonym_word = row[SYNONYM_COL]
    antonym_word = row[ANTONYM_COL]
    interpretion = row[INTERPRETION_COL]

    if synonym_word and type(synonym_word) != float:
        synonym_and_antonym_instructions.append({
            "category": task_category_mappings[SYNONYM_COL],
            "instruction": tasks_instructions[SYNONYM_COL].format(split_and_get_len(synonym_word, SEP)), 
            "input": word_name,
            "output": synonym_word
        })

    if antonym_word and type(antonym_word) != float:
        # print(antonym_word, SEP)
        synonym_and_antonym_instructions.append({
            "category": task_category_mappings[ANTONYM_COL],
            "instruction": tasks_instructions[ANTONYM_COL].format(split_and_get_len(antonym_word, SEP)), 
            "input": word_name,
            "output": antonym_word
        })

    MOE_instruction_dataset = pd.concat([MOE_instruction_dataset, pd.DataFrame.from_records(synonym_and_antonym_instructions)])


100%|██████████| 18264/18264 [09:16<00:00, 32.82it/s]


In [41]:
MOE_instruction_dataset

,category,instruction,input,output
0,translate_from_SC,翻譯成中文,1.介于七与九之间的自然数。如：「六、七、八、九……」。大写作「捌」，阿拉伯数字作「8」。2...,1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」，阿拉伯數字作「8」。2...
1,interpret,「六、七、八、九……。大寫作捌，阿拉伯數字作8」裡面的「八」是什麼意思,,名詞，介於七與九之間的自然數
3,interpret,「四通八達」裡面的「八」是什麼意思,,名詞，二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面
4,translate_from_SC,翻譯成中文,二一四部首之一。[形]表示数量是八的。如：「八字」、「八方」。[副]形容多数或多方面。如：「...,二一四部首之一。[形]表示數量是八的。如：「八字」、「八方」。[副]形容多數或多方面。如：「...
5,translate_from_SC,翻譯成中文,1.掘、挖。如：「扒土」、「扒洞」、「他在地里扒出土豆儿来。」2.拾。《西游记》第一四回：「...,1.掘、挖。如：「扒土」、「扒洞」、「他在地裡扒出土豆兒來。」2.拾。《西遊記》第一四回：「...
...,...,...,...,...
0,antonym,舉出1個反義詞,用武,媾和
0,synonym,舉出1個近似詞,佣鈿,回佣
0,synonym,舉出1個近似詞,佣錢,回佣
0,antonym,舉出1個反義詞,北迴歸線,南回歸線


In [42]:
MOE_instruction_dataset.to_csv("./instruction_datasets/ministry_of_education_revised_dictionary.csv", index=0)